In [241]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [291]:
# 데이터 불러오기
unsold_house = pd.read_excel('./1_데이터 재수집/규모별_미분양현황_20230218162415.xlsx', index_col = 0)
apt_rent = pd.read_excel('./1_데이터 재수집/아파트전세가격지수_시도_시_군_구__20230218162737.xlsx', index_col = 0)
birth = pd.read_excel('./1_데이터 재수집/출생아수_시도_시_군_구__20230218162012.xlsx', sheet_name = '1997-2022.11', index_col = 0)
unemployment_rate = pd.read_excel('./1_데이터 재수집/행정구역_시도__성별_실업률_20230218162056.xlsx', index_col = 0)

### 미분양 현황

In [243]:
# 필요없는 열 삭제
unsold_house.drop(['시도(1)'], inplace = True)

# index, col name 삭제 및 전치
unsold_house.index.name = '시점'
unsold_house.columns.name = None
unsold_house = unsold_house.T

In [244]:
# 연, 월의 데이터 구조 변환 : 2006.01 --> 2006-01-01로 진행

# index를 열로 추가
unsold_house['시점'] = unsold_house.index
unsold_house

시점,전국,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주,시점
2007.01,75616,697,8680,9467,243,7005,819,1180,-,3604,5312,2548,7449,5179,5374,6210,11772,77,2007.01
2007.02,73619,590,8653,9134,312,6246,803,1210,-,3361,5344,2645,7363,5098,5253,5948,11563,96,2007.02
2007.03,73162,687,8548,9189,266,5905,1048,1687,-,2579,5801,2565,7446,5184,4887,5854,11328,188,2007.03
2007.04,73393,685,8074,9009,219,5829,1020,1521,-,2488,5460,3632,8250,4812,4664,5419,12149,162,2007.04
2007.05,78571,704,7964,10888,362,7842,957,1500,-,2488,5212,4012,9195,4625,4640,6303,11723,156,2007.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.08,32722,610,1799,8301,1222,198,668,775,8,3180,1348,619,1386,157,2503,6693,2042,1213,2022.08
2022.09,41604,719,1973,10539,1541,163,1430,1426,5,5553,1262,1107,2418,621,2627,6520,2401,1299,2022.09
2022.10,47217,866,2514,10830,1666,161,1374,1414,6,5080,2287,1732,2840,1383,2797,6369,4176,1722,2022.10
2022.11,58027,865,2574,11700,2471,161,1853,2999,6,7037,2365,2632,5046,1951,2925,7667,4076,1699,2022.11


In [245]:
# 열 이름 변경
columns = ['전국', '서울특별시', '부산광역시', '대구광역시', '인천광역시', '광주광역시', '대전광역시', '울산광역시', 
           '세종특별자치시', '경기도', '강원도', '충청북도', '충청남도', '전라북도', '전라남도', '경상북도', '경상남도', '제주특별자치도', '시점']
unsold_house = pd.DataFrame(data = unsold_house.values, columns = columns)
unsold_house

,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,시점
0,75616,697,8680,9467,243,7005,819,1180,-,3604,5312,2548,7449,5179,5374,6210,11772,77,2007.01
1,73619,590,8653,9134,312,6246,803,1210,-,3361,5344,2645,7363,5098,5253,5948,11563,96,2007.02
2,73162,687,8548,9189,266,5905,1048,1687,-,2579,5801,2565,7446,5184,4887,5854,11328,188,2007.03
3,73393,685,8074,9009,219,5829,1020,1521,-,2488,5460,3632,8250,4812,4664,5419,12149,162,2007.04
4,78571,704,7964,10888,362,7842,957,1500,-,2488,5212,4012,9195,4625,4640,6303,11723,156,2007.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,32722,610,1799,8301,1222,198,668,775,8,3180,1348,619,1386,157,2503,6693,2042,1213,2022.08
188,41604,719,1973,10539,1541,163,1430,1426,5,5553,1262,1107,2418,621,2627,6520,2401,1299,2022.09
189,47217,866,2514,10830,1666,161,1374,1414,6,5080,2287,1732,2840,1383,2797,6369,4176,1722,2022.10
190,58027,865,2574,11700,2471,161,1853,2999,6,7037,2365,2632,5046,1951,2925,7667,4076,1699,2022.11


In [246]:
# 데이터프레임 형태에서 데이터 구조 통일화 : yyyy.1만 --> yyyy.10으로 변경
unsold_house['시점'] =  unsold_house['시점'].apply(lambda r : r + '0' if len(r) == 6 else r)
unsold_house.set_index('시점', inplace = True)

# 시점 열을 index로 만들어 datetime 형태 진행
unsold_house.index = pd.to_datetime(unsold_house.index, format = '%Y-%m')
unsold_house['세종특별자치시'].replace('-', 0, inplace = True)
unsold_house = unsold_house.apply(pd.to_numeric)
unsold_house.columns.name = None
unsold_house.to_excel('2007-2022_미분양현황.xlsx')

### 아파트(전세)가격지수

In [247]:
apt_rent.index.name = '시점'
apt_rent.columns.name = None
apt_rent = apt_rent.T

apt_rent.index = pd.to_datetime(apt_rent.index, format = '%Y-%m')
apt_rent['세종특별자치시'].replace('-', 0, inplace = True)
apt_rent = apt_rent.apply(pd.to_numeric)
apt_rent.columns.name = None

apt_rent.to_excel('2003-2022_아파트전세_가격지수.xlsx')

### 총 출생아수

In [292]:
birth.index.name = '시점'
birth.columns.name = None
birth = birth.T

# index를 열로 추가
birth['시점'] = birth.index
birth['시점'] = birth['시점'].astype('str')

# 데이터프레임 형태에서 데이터 구조 통일화 : yyyy.1만 --> yyyy.10으로 변경
birth['시점'] =  birth['시점'].apply(lambda r : r + '0' if len(r) == 6 else r)
birth

시점,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도,시점
1997.01,63268,13205,4201,3345,3646,2166,2003,1754,-,13440,1794,2075,2259,2579,2685,3362,4020,734,1997.01
1997.02,58144,12239,3910,2994,3193,1910,1869,1569,-,12334,1648,1798,2082,2261,2547,3257,3796,737,1997.02
1997.03,62160,13358,4304,3337,3466,2013,1894,1771,-,13058,1831,1988,2205,2364,2457,3343,4021,750,1997.03
1997.04,56949,12146,3872,3025,3189,1695,1666,1569,-,12101,1670,1750,2063,2203,2348,3183,3800,669,1997.04
1997.05,55270,11637,3826,2855,3074,1799,1765,1479,-,11644,1650,1734,1966,2085,2290,3024,3716,726,1997.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022.07,20441,3525,1138,818,1143,575,613,435,239,6274,590,600,864,608,693,881,1157,288,2022.07
2022.08,21758,3664,1251,857,1263,642,769,447,236,6619,631,674,926,576,688,976,1182,356,2022.08
2022.09,21885,3640,1200,860,1261,678,708,486,269,6652,653,668,887,590,767,985,1270,313,2022.09
2022.1,20658,3620,1159,848,1204,633,659,445,276,6152,613,585,850,575,640,961,1162,278,2022.10


In [293]:
birth.set_index('시점', inplace = True)
birth.index = pd.to_datetime(birth.index, format = '%Y-%m')
birth['세종특별자치시'].replace('-', 0, inplace = True)
birth = birth.apply(pd.to_numeric)
birth.columns.name = None
birth

,전국,서울특별시,부산광역시,대구광역시,인천광역시,광주광역시,대전광역시,울산광역시,세종특별자치시,경기도,강원도,충청북도,충청남도,전라북도,전라남도,경상북도,경상남도,제주특별자치도
시점,,,,,,,,,,,,,,,,,,
1997-01-01,63268,13205,4201,3345,3646,2166,2003,1754,0,13440,1794,2075,2259,2579,2685,3362,4020,734
1997-02-01,58144,12239,3910,2994,3193,1910,1869,1569,0,12334,1648,1798,2082,2261,2547,3257,3796,737
1997-03-01,62160,13358,4304,3337,3466,2013,1894,1771,0,13058,1831,1988,2205,2364,2457,3343,4021,750
1997-04-01,56949,12146,3872,3025,3189,1695,1666,1569,0,12101,1670,1750,2063,2203,2348,3183,3800,669
1997-05-01,55270,11637,3826,2855,3074,1799,1765,1479,0,11644,1650,1734,1966,2085,2290,3024,3716,726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-01,20441,3525,1138,818,1143,575,613,435,239,6274,590,600,864,608,693,881,1157,288
2022-08-01,21758,3664,1251,857,1263,642,769,447,236,6619,631,674,926,576,688,976,1182,356
2022-09-01,21885,3640,1200,860,1261,678,708,486,269,6652,653,668,887,590,767,985,1270,313


In [294]:
birth.to_excel('1997-2022_총_출생아수.xlsx')

<hr>

In [ ]:
모델 찾으면 --> 전국, 서울시 ~ 제주도 모두 돌려보기
못 찾으면 --> 전국, 서울시, 세종시만 돌리기
